### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import pandas as pd
from datetime import datetime 

# Read Ironore_price_market_index_test.csv File and store into Pandas data frame
commodity_to_load = "Resources/Ironore_price_market_index_test.csv"
purchase_data = pd.read_csv(commodity_to_load)
#purchase_data

In [2]:
#remove empty columns, fix naming error and bring Date in line with other dataframes
clean_data = purchase_data[['Date','Iron Price','Silver Price','Gold Price',' Copper Price','S&P GSCI Price']]
clean_data = clean_data.rename(columns={' Copper Price': 'Copper Price'})
clean_data['Date'] = clean_data['Date'].str[2:]
#clean_data

#save data to Commodities_clean.csv
clean_data.to_csv('Outputs/Commodities_clean.csv')

In [16]:
#https://www.kaggle.com/docs/api
#https://stackoverflow.com/questions/64393520/r-not-working-as-lineterminator-within-python-csv-writer
# Read all_perth_310121.csv File and store into Pandas data frame
perth_to_load = "Resources/all_perth_310121.csv"

perth_data = pd.read_csv(perth_to_load)


In [17]:
#fix up Date_sold format errors and bring in line with other datasets
perth_data["DATE_SOLD"] = perth_data["DATE_SOLD"].str.replace('\r','')
perth_data["DATE_SOLD"] = perth_data["DATE_SOLD"].str.replace('-','/')
#perth_data

In [5]:
#Limit dataframe to variables of interest
clean_perth = perth_data[['SUBURB',
                          'PRICE',
                          'BEDROOMS',
                          'BATHROOMS',
                          'GARAGE',
                          'LAND_AREA',
                          'FLOOR_AREA',
                          'DATE_SOLD',
                          'POSTCODE',
                          'LATITUDE',
                          'LONGITUDE']]
clean_perth = clean_perth.rename(columns={'DATE_SOLD': 'Date'})
#Add unique ID and add City for later merging with Sydney data
clean_perth.insert(0, 'ID', range(1, 1 + len(clean_perth)))
clean_perth['CITY'] = "Perth"
#clean_perth

In [6]:
#merge Perth data with Commodities data 
Perth_commodities_df = pd.merge(clean_data, clean_perth, on="Date",how="inner")
#Perth_commodities_df

In [7]:
#save data to perth_clean.csv
Perth_commodities_df.to_csv('Outputs/perth_clean.csv')

In [8]:
# Read SydneyHousePrices.csv File and store into Pandas data frame
Sydney_to_load = "SydneyHousePrices.csv"
Sydney_data = pd.read_csv(Sydney_to_load)

In [9]:
#Add City indicator for when merged with Perth data
Sydney_data["CITY"] = "Sydney"
#sanity check to see earliest date
#Sydney_data.sort_values(by=['Date'])


In [10]:
#Change the Date from string to datetime then convert to desired format 
Sydney_data['Date'] = pd.to_datetime(Sydney_data['Date'], errors='coerce')
Sydney_data['Date'] = Sydney_data["Date"].dt.strftime("%m/20%y")
#Sydney_data

In [11]:
#Rename columns matching Perth data to combine later
clean_Sydney = Sydney_data.rename(columns={'Id':'ID',
                          'suburb':'SUBURB',
                          'sellPrice':'PRICE',
                          'bed':'BEDROOMS',
                          'bath':'BATHROOMS',
                          'car':'GARAGE',
                          'postalCode':'POSTCODE'})
#clean_Sydney

In [12]:
#merge commodity prices with Sydney data
sydney_commodities_df = pd.merge(clean_data, clean_Sydney, on="Date",how="inner")
#sydney_commodities_df
#save data to sydney_clean.csv
sydney_commodities_df.to_csv('Outputs/sydney_clean.csv')

In [13]:
#combine Perth and Sydney data into one DataFrame
complete_city_df = pd.concat([clean_Sydney, clean_perth], ignore_index=True)
#limit variables to match eachother
complete_city_df = complete_city_df[[
                          'SUBURB',
                          'PRICE',
                          'BEDROOMS',
                          'BATHROOMS',
                          'GARAGE',
                          'Date',
                          'POSTCODE',
                           'CITY']]

#merge commodity prices with city data and add unique ID for each sale
complete_df = pd.merge(clean_data, complete_city_df, on="Date",how="inner")
complete_df.insert(0, 'ID', range(1, 1 + len(complete_df)))
#complete_df

In [14]:
#save data to complete_house_commodities.csv
complete_df.to_csv('Outputs/complete_house_commodities.csv')